In [29]:
import re
from metagpt.actions import Action
from metagpt.logs import logger
import asyncio

In [33]:
class ExtractName(Action):
    PROMPT_TEMPLATE: str = """
    Extract the name of the restaurant from "{query}" and compare it to the following list of names: {names}, 
    and return the name that is most similar to the extracted name. 
    Return ```NAME: the selected name from the list``` with NO other texts.
    """

    name: str = "ExtractName"

    async def run(self, query: str, names: list):
        prompt = self.PROMPT_TEMPLATE.format(query=query, names=names)

        rsp = await self._aask(prompt)

        extracted_name = ExtractName.extract_name(rsp)
    
        return extracted_name

    @staticmethod
    def extract_name(rsp):
        pattern = r"NAME: (.*)"
        match = re.search(pattern, rsp)
        name = match.group(1) if match else rsp
        return name.strip()

In [34]:
names = (
    "McDonald's, Subway, Taco Bell, Chick-fil-A, Applebee's, Olive Garden, Cheesecake Factory, "
    "Buffalo Wild Wings, Starbucks, Krispy Kreme, Panera Bread, Tim Horton's, Chipotle, In-n-Out, "
    "Five Guys, Panda Express, Pret A Manger, Cinnabon, IHOP, Burger King"
)

query="What is the overall score for Chickfil/A?"

role = ExtractName()
logger.info(query)
result = asyncio.run(role.run(query, names))

print(result)

2024-11-07 13:39:10.453 | INFO     | __main__:<module>:10 - What is the overall score for Chickfil/A?
2024-11-07 13:39:11.106 | WARNING  | metagpt.provider.openai_api:_calc_usage:244 - usage calculation failed: num_tokens_from_messages() is not implemented for model gpt-4o-mini. See https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken for information on how messages are converted to tokens.


NAME: Chick-fil-A
Chick-fil-A
